Import libs

In [8]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests

import os
import time
import glob
import itertools
import lxml

Generator function  for downloading pages

In [4]:
def post_from_url_generator(base_url, pages=999999, sleep_time=5):
    i = 1
    resp = "rest"
    while resp != "\"\"" and i <= pages:
        url = f"{base_url}{i}"
        print(f"Getting POST response from URL: {url}")
        resp = "testestes"
        resp = requests.post(url)
        time.sleep(sleep_time)
        yield resp.content
        i += 1

Download pages

In [5]:
base_url = "https://blog.prokulski.science/index.php/wp-json/nv/v1/posts/page/"
gen = post_from_url_generator(base_url, pages=5)
#list(gen)


Extract post pages from url's

In [6]:
def get_post_urls_from_page(page):
    soup = BeautifulSoup(page, 'html.parser')
    result_urls = []
    for x in soup.select("h2 > a"):
        url = str(x['href']).replace("\\\"", "").replace("\\", "")
        result_urls.append(url)
    return result_urls


In [7]:
base_url = "https://blog.prokulski.science/index.php/wp-json/nv/v1/posts/page/"

post_urls = [get_post_urls_from_page(page) for page in post_from_url_generator(base_url, pages=3)]
post_urls = list(itertools.chain.from_iterable(post_urls))

Getting POST response from URL: https://blog.prokulski.science/index.php/wp-json/nv/v1/posts/page/1


KeyboardInterrupt: 

In [ ]:
post_urls


Download all post's web pages and save local

In [ ]:
for url in post_urls:
    post_page = requests.get(url).content
    post_title = url.replace("https://blog.prokulski.science/index.php/", "").replace("/", "_")[:-1]
    print(f'Saving post page in file: {post_title}.html')
    with open("pages/"+post_title+".html", "wb") as outfile:
        outfile.write(post_page)



Load web pages from files and return list of BeautifulSoap objects

In [9]:
def read_posts_from_disk(path: str, parser='html.parser'):
    files = glob.glob(path)
    posts = []

    for file in files:
        with open(file, "rb") as input_file:
            filename = os.path.basename(file)
            post_date = filename[:10].replace("_", "-")
            post_name = filename.replace(".html", "")
            soup = BeautifulSoup(input_file.read(), parser)
            posts.append((post_date, post_name, soup))
    return posts

### Load pages from local file system

In [24]:
posts = read_posts_from_disk("pages/2005*", parser='lxml')

### Functions for extract some attributes from BeatutifulSoup web page

In [34]:
def get_tags(soup):
    tag_links = soup.select(".nv-tags-list > a")
    return list(map(lambda x: x.string, tag_links))


def get_listings_count(soup) -> int:
    code_divs = soup.select(".crayon-code > .crayon-pre")
    return len(code_divs)


def get_code_lines_count(soup) -> int:
    line_divs = soup.select(".crayon-code > .crayon-pre > .crayon-line")
    return len(line_divs)


def get_tables_count(soup) -> int:
    tables = soup.select("table:is(.table, .table-striped, .table-hover, .table-condensed, .table-responsive)")
    return len(tables)


def get_images_count(soup):
    imgs = soup.select("div.nv-content-wrap.entry-content")[0].findChildren("img")
    return len(imgs)


def get_comments(soup):
    comments = []
    for comment in soup.select(".nv-comment-article")[:2]:
        comment_row = {}
        comment_row['author'] = comment.select(".comment-author .author")[0].get_text()
        # print(comment.body.author.get_text())
        comment_row['date'] = pd.to_datetime(comment.select("time.entry-date.published")[0]
                                     .text.replace(" o", ""))
        content_tags = comment.select("div.nv-comment-content.comment.nv-content-wrap > p")[0]
        comment_row['comment'] = content_tags.get_text()
        comments.append(comment_row)
    return comments

In [35]:
get_comments(posts[4][2])

[{'author': 'Przemek',
  'date': Timestamp('2005-02-20 22:11:00'),
  'comment': 'Do tej pory używałem starego Zenita ojca, a ostatnio nawet bawiłem się Smieną, ale na odbitkach można faktycznie zbankrutować. Dorobiłem się niedawno Nikona  3100 – może nie jest to szczyt marzeń, ale przy ładnej pogodzie fotki robią wrażenie, a to chyba najważniejsze, no i można pstrykać do woli.'},
 {'author': 'olympus',
  'date': Timestamp('2007-09-19 19:37:00'),
  'comment': 'Ciekawy jestem kiedy przesiądziesz na lustrzankę'}]

In [28]:
# posts[0]




[{'author': 'Przemek',
  'date': Timestamp('2005-02-20 22:11:00'),
  'comment': 'Do tej pory używałem starego Zenita ojca, a ostatnio nawet bawiłem się Smieną, ale na odbitkach można faktycznie zbankrutować. Dorobiłem się niedawno Nikona  3100 – może nie jest to szczyt marzeń, ale przy ładnej pogodzie fotki robią wrażenie, a to chyba najważniejsze, no i można pstrykać do woli.'},
 {'author': 'olympus',
  'date': Timestamp('2007-09-19 19:37:00'),
  'comment': 'Ciekawy jestem kiedy przesiądziesz na lustrzankę'}]

In [ ]:
# posts[0]

Convert BeautifulSoup objects to dataframe and extract features from web page

In [38]:
rows_list = []
comments_list = []
i=1
for date, title, soup in posts:
    row = {}
    row['id'] = i
    row['title'] = title
    row['post_date'] = date
    row['code_lines'] = get_code_lines_count(soup)
    row['listings_num'] = get_listings_count(soup)
    row['tables_count'] = get_tables_count(soup)
    row['tags'] = get_tags(soup)
    row['comments'] = get_comments(soup)
    rows_list.append(row)

    comm = {'id': i, 'comments': get_comments(soup)}
    comments_list.append(comm)
    i += 1

post_df = pd.DataFrame(rows_list)
comment_df = pd.DataFrame(comments_list)

In [39]:
post_df.head()

,title,post_date,code_lines,listings_num,tables_count,tags,comments
0,2005_01_04_na-dobry-pocztek,2005-01-04,0,0,0,"[blog, domena, host, hosting, partia]","[{'author': 'Michalski9', 'date': 2019-02-12 0..."
1,2005_01_04_pink-floyd-nieznany,2005-01-04,0,0,0,[],"[{'author': 'vera', 'date': 2006-09-30 10:57:0..."
2,2005_01_05_pink-floyd-nieznany-cig-dalszy,2005-01-05,0,0,0,[partia],[]
3,2005_01_11_marzyciel,2005-01-11,0,0,0,[],[]
4,2005_01_11_olympus-c-765,2005-01-11,0,0,0,"[kraków, olympus, zdjęcia]","[{'author': 'Przemek', 'date': 2005-02-20 22:1..."


In [ ]:
comment_df.head()